In [15]:
## Importar fitxer SMILES des de SABIO

In [2]:
#install
import requests
import pandas as pd 

from urllib.request import urlopen
from urllib.parse import quote

def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'


import os.path 

datafile = "data/smiles.csv"

if os.path.exists(datafile): 
    print(datafile, " exists") 
else: 
    print("Generating ",datafile)

    QUERY_URL = 'http://sabiork.h-its.org/sabioRestWebServices/searchReactionParticipants'

    query = {"SabioReactionID":"*", "fields[]":["Name","Role", "Smiles"]}

    request = requests.post(QUERY_URL, params = query) 
    request.raise_for_status() 

    print(request.text)

    # results

    fh=open("data/smiles.csv", "w")

    fh.write(request.text)







data/smiles.csv  exists


## crear un dataframe a partir del firtxer SMILES

In [3]:
df = pd.read_csv(datafile, sep='\t')
print(df)


       SabioReactionID                              Name       Role  \
0                    1       alpha-D-Glucose 6-phosphate  Substrate   
1                    1       beta-D-Fructose 6-phosphate    Product   
2                    2                               ATP  Substrate   
3                    2  beta-D-Fructose 1,6-bisphosphate    Product   
4                    2                               ADP    Product   
...                ...                               ...        ...   
62291            16513                                H+    Product   
62292            16513                                O2  Substrate   
62293            16513                                CO    Product   
62294            16513      2-Heptyl-3-hydroxy-quinolone  Substrate   
62295            16513        n-Octanoylanthranilic acid    Product   

                                                  Smiles  
0      C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)O...  
1      C([C@@H]1[C@H]([C@@H](

## Generem un DF curt per a fer proves

Crear smiles reacció:

In [4]:
input='data/smilestest.csv'
output='data/ReactionSmilesTest.csv'

df = pd.read_csv(input, sep='\t')


df['Smiles'] = df['Smiles'].fillna(0)

# create an empty list  
list1 = [] 
#smiles=row["Smiles"] 
# iterate over each row of dataframe   
for i in range(len(df)): 

    # check if Smiles is 0 then append the SabioreactionID to list1  
    if df.loc[i,'Smiles'] == 0: 
        list1.append(df.loc[i,'SabioReactionID']) 

df = df.drop(df[df['SabioReactionID'].isin(list1)].index) 
# Print the updated dataframe 
print(df)


#Generem un dataframe buit amb les columnes SabioID i Reaction de 11 files.
df = df.reset_index()  #la funció rest_index() restableix l'índex del dataframe, estableix una llista del 0 fins a la longitud del dataframe
numrows=11
df2 = pd.DataFrame(columns=["SabioID", "Reaction"]) #, index=range(numrows))
#print(df2)

#Creem dues llistes buides amb tants strings com files (en aquest cas 11 numrows=11). Generem una llista R per guardar els reactius i una llista P per guardar els productes.
R=[""]*(numrows+1)
P=[""]*(numrows+1)
isFirstR=[True]*(numrows+1)
isFirstP=[True]*(numrows+1)

def check_smiles(smiles):
    print(smiles)
    if not smiles:
        print('Wrong SMILES for:',smiles)
        return 0
    return 1


#Fem una iteració per cada fila del dataframe amb el command: for index, row in df.iterrows():

for index, row in df.iterrows():
    rid=row['SabioReactionID']
    print('RID:',rid,isFirstR[rid],isFirstP[rid])
    #irid=int(rid)
    #print(rid)
    #print(type(rid))
    
    name=row["Name"]
    smiles=row["Smiles"]
    if check_smiles(smiles) == 0:
        break
   
    role=row["Role"]
    if role == "Substrate": 
        if isFirstR[rid]:
            R[rid]=R[rid] + smiles 
            isFirstR[rid]= False
        else:
            R[rid]=R[rid] + "." + smiles
            
        
    elif role == "Product":
        if isFirstP[rid]:
            P[rid]=P[rid]+smiles
            isFirstP[rid]=False
        else:
            P[rid]=P[rid] +"."+ smiles
    else:
        print("error, name")
    print(R[rid],P[rid])
    df2.at[rid, "SabioID"]=rid
    
for i in range(numrows):
    print(R[i]+">"+P[i])
    df2.at[i, "Reaction"]=R[i]+">"+P[i]

  

df2.drop(df2.tail(2).index,inplace=True)
print(df2)

df2.to_csv(output)


    SabioReactionID                              Name       Role  \
0                 1       alpha-D-Glucose 6-phosphate  Substrate   
1                 1       beta-D-Fructose 6-phosphate    Product   
2                 2                               ATP  Substrate   
3                 2  beta-D-Fructose 1,6-bisphosphate    Product   
4                 2                               ADP    Product   
5                 2       beta-D-Fructose 6-phosphate  Substrate   
6                 3      D-Glyceraldehyde 3-phosphate    Product   
7                 3               Glycerone phosphate    Product   
8                 3  beta-D-Fructose 1,6-bisphosphate  Substrate   
9                 4      D-Glyceraldehyde 3-phosphate  Substrate   
10                4               Glycerone phosphate    Product   
11                5                                H+    Product   
12                5                              NADH    Product   
13                5      D-Glyceraldehyde 3-phos

# RDKit

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from rdkit import Chem

dfRS = pd.read_csv('data/ReactionSmilesTest.csv')

# Create a new column in the dataframe to hold the RDKit molecule objects
dfRS['Mol'] = None

# Iterate over the rows in the dataframe and create RDKit molecule objects
for i, row in dfRS.iterrows():
    smiles = row['Reaction']  
    mol = Chem.MolFromSmiles(smiles)
    df.at[i, 'Mol'] = mol





[20:00:32] SMILES Parse Error: syntax error while parsing: C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)OP(=O)(O)O>C([C@@H]1[C@H]([C@@H]([C@](O1)(CO)O)O)O)OP(=O)(O)O
[20:00:32] SMILES Parse Error: Failed parsing SMILES 'C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)OP(=O)(O)O>C([C@@H]1[C@H]([C@@H]([C@](O1)(CO)O)O)O)OP(=O)(O)O' for input: 'C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)OP(=O)(O)O>C([C@@H]1[C@H]([C@@H]([C@](O1)(CO)O)O)O)OP(=O)(O)O'
[20:00:32] SMILES Parse Error: syntax error while parsing: Nc3ncnc1c3(ncn1C2OC(COP(O)(=O)OP(O)(=O)OP(O)(O)=O)C(O)C2(O)).C([C@@H]1[C@H]([C@@H]([C@](O1)(CO)O)O)O)OP(=O)(O)O>OC1C(COP(O)(O)=O)OC(O)(COP(O)(O)=O)C1(O).Nc3ncnc1c3(ncn1C2OC(COP(O)(=O)OP(O)(O)=O)C(O)C2(O))
[20:00:32] SMILES Parse Error: Failed parsing SMILES 'Nc3ncnc1c3(ncn1C2OC(COP(O)(=O)OP(O)(=O)OP(O)(O)=O)C(O)C2(O)).C([C@@H]1[C@H]([C@@H]([C@](O1)(CO)O)O)O)OP(=O)(O)O>OC1C(COP(O)(O)=O)OC(O)(COP(O)(O)=O)C1(O).Nc3ncnc1c3(ncn1C2OC(COP(O)(=O)OP(O)(O)=O)C(O)C2(O))' for input: 'Nc3ncnc1c3(ncn1C